<a href="https://colab.research.google.com/github/kangmyoungseok/updater/blob/main/updater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. pair.py


실행 -> pairs_v1.x.csv파일 나옴

In [ ]:
from pprint import pprint
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pandas as pd
import json
from bs4 import BeautifulSoup
import re # 추가
from urllib.request import urlopen
import requests
import time

# swap function makes scam token to token0
def switch_token(result):
    for pair in result['data']['pairs']:
        if (int(pair['token0']['txCount']) > int(pair['token1']['txCount'] )):
            pair['reserve00'],pair['reserve11'] = pair['reserve1'],pair['reserve0']
            pair['token00'],pair['token11'] = pair['token1'],pair['token0']
        else:
            pair['reserve00'],pair['reserve11'] = pair['reserve0'],pair['reserve1']
            pair['token00'],pair['token11'] = pair['token0'],pair['token1']
    
# function to use requests.post to make an API call to the subgraph url
def run_query(query):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
                            '',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))


query_init = '''
{
 pairs(first: 1000, orderBy: createdAtBlockNumber, orderDirection: desc) {
   id
   token0{
    id
    symbol
    name
    txCount
    totalLiquidity
  }
   token1{
    id
    symbol
    name
    txCount
    totalLiquidity
  }
   reserve0
   reserve1
   totalSupply
   reserveUSD
   reserveETH
   txCount
   createdAtTimestamp
   createdAtBlockNumber
 }
}
''' 


query_iter = '''
{
 pairs(first: 1000, orderBy: createdAtBlockNumber, orderDirection: desc, where: {createdAtBlockNumber_lt:initial}) {
   id
   token0{
    id
    symbol
    name
    txCount
    totalLiquidity
  }
   token1{
    id
    symbol
    name
    txCount
    totalLiquidity
  }
   reserve0
   reserve1
   totalSupply
   reserveUSD
   reserveETH
   txCount
   createdAtTimestamp
   createdAtBlockNumber
 }
}
''' 

pair_frame = [] # 쿼리의 결과를 여기에 List 형태로 담을 것. 50000개

##### 맨 처음 쿼리. 반복문 불가####
query = query_init
result = run_query(query)
switch_token(result)
for pair in result['data']['pairs']:
    if((pair['token0']['symbol'] != 'WETH') and (pair['token1']['symbol'] !='WETH' )):
      continue
    if((pair['token00']['txCount'] == 0) or (pair['token00']['txCount'] == '0')):
      continue
    year = time.gmtime(int(pair['createdAtTimestamp'])).tm_year
    month = time.gmtime(int(pair['createdAtTimestamp'])).tm_mon
    day = time.gmtime(int(pair['createdAtTimestamp'])).tm_mday
    pair['createdAtTimestamp'] = str(year) + '-' + str(month) + '-' + str(day)
    pair_frame.append(pair)

last_block = result['data']['pairs'][999]['createdAtBlockNumber']
query_iter = query_iter.replace('initial',last_block)
query = query_iter

try:
    while(1):
        result = run_query(query_iter)
        switch_token(result)
        print(result.keys())
        for pair in result['data']['pairs']:
            if((pair['token0']['symbol'] != 'WETH') & (pair['token1']['symbol'] !='WETH' )):
              continue
            if((pair['token00']['txCount'] == 0) or (pair['token00']['txCount'] == '0')):
              continue
            year = time.gmtime(int(pair['createdAtTimestamp'])).tm_year
            month = time.gmtime(int(pair['createdAtTimestamp'])).tm_mon
            day = time.gmtime(int(pair['createdAtTimestamp'])).tm_mday
            pair['createdAtTimestamp'] = str(year) + '-' + str(month) + '-' + str(day)
            pair_frame.append(pair)
        query_iter = query_iter.replace(last_block,result['data']['pairs'][999]['createdAtBlockNumber'])
        last_block = result['data']['pairs'][999]['createdAtBlockNumber']
        print(last_block)

except Exception as e:
    try:
      print(result['errors'])
    except:
      print(e)
    df = pd.json_normalize(pair_frame)
    df.to_csv('Pairs_v1.5.csv',encoding='utf-8-sig',index=False)





#2. Uniswap_API.py 파일

input = pairs_v1.x 

output = mint.json / burn.json / swap.json

In [ ]:
from pprint import pprint
from pandas.core.frame import DataFrame
import pandas as pd
import json
import requests
import time
from tqdm import tqdm
from multiprocessing import Pool
from multiprocessing import Process
# function to use requests.post to make an API call to the subgraph url

global df
global df_len
df = pd.read_csv('./Pairs_v1.5.csv').to_dict()
df_len = len(df['id'])
global count
count= 0

def run_query(query):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
                            '',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))


mint_query_template = '''
{
  mints(orderBy: timestamp, orderDirection: asc, where:{ pair: "pair_address" }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
''' 

swap_query_template = '''
{
  swaps(orderBy: timestamp, orderDirection: asc, where:{ pair: "pair_address" }) {
      amount0In
      amount0Out
      amount1In
      amount1Out
      to
      sender
      timestamp
 }
}
''' 

burn_query_template = '''
{
  burns(orderBy: timestamp, orderDirection: asc, where:{ pair: "pair_address" }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
''' 

#############모든 pair 쌍에 대해서 Mint Query 후 결과 저장##############
def get_mint_subProcess(pair_address):
    try:
        query = mint_query_template.replace('pair_address',pair_address)
        result = run_query(query)
        return {pair_address : result['data']['mints']}
    except:
        return {pair_address : ['Error Occur']}
        
def get_mint():
    file_path = "./mint.json"
    mint_json = {}
    try:
      p = Pool(8)
      start = time.time()
      global count
      for ret in p.imap(get_mint_subProcess,df['id'].values()):
        count = count+1
#        print("Got value",ret,"Time :",time.time()-start)
        mint_json.update(ret) 
        if(count % 1000 == 0):
          print("Process Rate : {}/{} {}%".format(count,df_len,int((count/df_len)*100)))
#          print("write file count : " + str(count))
#          with open(file_path,'w') as outfile:
#            json.dump(mint_json, outfile, indent=4)    
  
      print('finish ' + str(count))
      with open(file_path,'w') as outfile:
            json.dump(mint_json, outfile, indent=4)
      delta_t = time.time() - start
      print("Total Time :",delta_t)
      p.close()
      p.join()
      mint_json.clear()        

    except Exception as e:
        print(e)


#############모든 pair 쌍에 대해서 Mint Query 후 결과 저장##############
def get_swap_subProcess(pair_address):
    try:
        query = swap_query_template.replace('pair_address',pair_address)
        result = run_query(query)
        return {pair_address : result['data']['swaps']}
    except:
        return {pair_address : ['Error Occur']}
        
def get_swap():
    file_path = "./swap.json"
    swap_json = {}
    try:
        p = Pool(8)
        start = time.time()
        global count
        for ret in p.imap(get_swap_subProcess,df['id'].values()):
            count = count+1
            swap_json.update(ret)
            if(count % 1000 == 0):
                print("Process Rate : {}/{} {}%".format(count,df_len,int((count/df_len)*100)))
 #               print("write file count : " + str(count))
 #               with open(file_path,'w') as outfile:
 #                   json.dump(swap_json, outfile, indent=4)    
  
        p.close()
        p.join()
        print('finish ' + str(count))
        with open(file_path,'w') as outfile:
            json.dump(swap_json, outfile, indent=4)
        delta_t = time.time() - start
        print("Total Time :",delta_t)
        swap_json.clear()        

    except Exception as e:
        print(e)

#############모든 pair 쌍에 대해서 Burn Query 후 결과 저장##############
def get_burn_subProcess(pair_address):
    try:
        query = burn_query_template.replace('pair_address',pair_address)
        result = run_query(query)
        return {pair_address : result['data']['burns']}
    except:
        return {pair_address : ['Error Occur']}
        
def get_burn():
    file_path = "./burn.json"
    burn_json = {}
    try:
        p = Pool(8)
        start = time.time()
        global count
        for ret in p.imap(get_burn_subProcess,df['id'].values()):
            count = count+1
            burn_json.update(ret)
            if(count % 1000 == 0):
                print("Process Rate : {}/{} {}%".format(count,df_len,int((count/df_len)*100)))
#                print("write file count : " + str(count))
#                with open(file_path,'w') as outfile:
#                    json.dump(burn_json, outfile, indent=4)    
  
        p.close()
        p.join()
        print('finish ' + str(count))
        with open(file_path,'w') as outfile:
            json.dump(burn_json, outfile, indent=4)
        delta_t = time.time() - start
        print("Total Time :",delta_t)
        burn_json.clear()        

    except Exception as e:
        print(e)


if __name__=='__main__': 
    get_mint()
    get_burn()
    get_swap()

#3. Initial_Liquidity_Eth.py

input = Pairs파일 / mint.json

initial_Liquidity ETH / Token 열 추가

In [9]:
from pandas.core.frame import DataFrame
from requests import Request, Session
import pandas as pd
import json
from bs4 import BeautifulSoup
import re # 추가
from urllib.request import urlopen
import requests
import time
from multiprocessing import Pool
import json


with open('/content/drive/MyDrive/mint.json', 'r') as f:
    mint_json = json.load(f)

len(mint_json)
datas = pd.read_csv('/content/drive/MyDrive/Pairs_v1.5.csv',encoding='utf-8-sig').to_dict('records')


for data in datas:
    pair_id = data['id']

    if(len(mint_json[pair_id]) == 0 ):     #Mint 정보가 없는 pair는 초기 Liquidity를 0으로 한다.
        data['initial_Liquidity_ETH'] = 0
        data['initial_Liquidity_token'] = 0
        continue


    if(data['token0.symbol'] == 'WETH'):
        initial_Liquidity_ETH = mint_json[pair_id][0]['amount0']
        initial_Liquidity_token = mint_json[pair_id][0]['amount1']
    else:
        initial_Liquidity_ETH = mint_json[pair_id][0]['amount1']
        initial_Liquidity_token = mint_json[pair_id][0]['amount0']

    data['initial_Liquidity_ETH'] = initial_Liquidity_ETH  
    data['initial_Liquidity_token'] = initial_Liquidity_token



DataFrame(datas).to_csv('Pairs_v1.5.csv',encoding='utf-8-sig')

#4.Creator.py

Creator 긁어 오는건 시간 많이 걸림.
이전 버전의 파일에서 Creator를 수동으로 열 추가해주고 없는 부분만 다시 찾아온다.




In [5]:
#Etherplorer API / Etherscan Crawling으로 Creator Address를 구한다.
from pandas.core.frame import DataFrame
from requests import Request, Session
import pandas as pd
import json
from bs4 import BeautifulSoup
import re # 추가
from urllib.request import urlopen
import requests
import time
from multiprocessing import Pool
import os
import glob
 
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 

def split_csv(total_csv):
    rows = pd.read_csv(total_csv,chunksize=5000)
    file_count = 0
    for i, chuck in enumerate(rows):
        chuck.to_csv('./result/out{}.csv'.format(i))
        file_count = file_count+1 
    return file_count

def merge_csv():
  input_file = r'./result/'
  output_file = r'./result/Pairs_v1.5.csv'

  allFile_list = glob.glob(os.path.join(input_file, 'fout*')) # glob함수로 sales_로 시작하는 파일들을 모은다
  allFile_list.sort()
  print(allFile_list)

  all_Data = []
  for file in allFile_list:
    records = pd.read_csv(file).to_dict('records') 
    all_Data.extend(records)

  DataFrame(all_Data).to_csv(output_file,encoding='utf-8-sig',index=False)
  
def get_creatorAddress(data):
    if(str(data['token00_creator_address']) != 'nan'):
        return data
    token_id = data['token00.id']
    repos_url = 'https://api.ethplorer.io/getAddressInfo/'+token_id+'?apiKey=EK-4L18F-Y2jC1b7-9qC3N'
    response = requests.get(repos_url).text
    repos = json.loads(response)    #json 형태로 token_id에 해당하는 정보를 불러온다.
    
    try:
        creator_address = repos['contractInfo']['creatorAddress']
        print('find by ethplorer : ' + token_id)
    except:     #오류가 나면 이더스캔에서 크롤링
         url = 'https://etherscan.io/address/'+token_id
         try:
             time.sleep(2)
             response = requests.get(url,headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'})
             page_soup = BeautifulSoup(response.text, "html.parser")
             Transfers_info_table_1 = str(page_soup.find("a", {"class": "hash-tag text-truncate"}))
             creator_address = re.sub('<.+?>', '', Transfers_info_table_1, 0).strip()
             print('find by etherscan : ' + token_id)
             print('result : ' + creator_address)
         except Exception as e:  #이더스캔 크롤링까지 에러나면 'Error'로 표시
              print(e)
              timeout_count = 0
              while(timeout_count<2):
                  try:
                      print('timeout in, address' + token_id)
                      time.sleep(20)
                      timeout_count = timeout_count + 1
                      response = requests.get(repos_url).text
                      repos = json.loads(response)
                      creator_address = repos['contractInfo']['creatorAddress']
                      break
                  except:
                      creator_address = 'Error'

    data['token00_creator_address'] = creator_address
    return data

if __name__=='__main__':
    createFolder('./result')
    file_name = 'Pairs_v1.5.csv'
    file_count = split_csv(file_name)
    out_list = []
    out_list = list(input('입력(공백단위) : ').split())

    for i in out_list:         #하나의 파일 단위로 Creator Address 불러오고, 해당 초기 유동성풀 이더값 구해온다.
        file_name = './result/out{}.csv'.format(i)
        datas = pd.read_csv(file_name).to_dict('records')
        datas_len = len(datas)
        try:
            p = Pool(4)
            count = 0
            result = []
            for ret in p.imap(get_creatorAddress,datas):
                count = count+1
                result.append(ret)
                if(count % 200 == 0):
                    print("Process Rate : {}/{} {}%".format(count,datas_len,int((count/datas_len)*100)))
            p.close()
            p.join()
        except Exception as e:
            print(e)
        print('=======')
        time.sleep(5)
            
        df = pd.DataFrame(result)
        file_name = './result/fout{}.csv'.format(i)
        df.to_csv(file_name,encoding='utf-8-sig',index=False)
        print(file_name + ' complete')
    merge_csv()
    

dict